Imports

In [1]:
import sys
sys.path.append('..')
import mdtraj

from src.workshop_2_utils import *

from config import settings

Set those variables!

In [2]:
# smina parameters
num_poses = 10
exhaustiveness = 20

In [3]:
# How many molecules we want to stuff for
num_mols = 5

data_path = settings.data_path / "lab2"

prepared_protein_pdb_file = data_path / "6vhn_prepared.pdb"
ligand_pdb_file = data_path / "ligand.pdb"

all_ligands_sdf_file = data_path / "Enamine_Hinge_Binders_Library_plated_24000cmds_20210316 (1).sdf"

save_df_file = data_path / "poses.parquet"

In [4]:

smina_inputs_path = data_path / "smina_inputs"
smina_inputs_path.mkdir(exist_ok=True)
smina_outputs_path = data_path / "smina_outputs"
smina_outputs_path.mkdir(exist_ok=True)

protein_pdbqt_path = smina_inputs_path / "receptor.pdbqt"


We prepare the pdbqt file for the protein

In [5]:
protein_pdbqt_file = smina_inputs_path / "receptor.pdbqt"

prep=Preprocessor()
prep.prepare_receptor(prepared_protein_pdb_file, protein_pdbqt_file)

Prepare the bounding box

In [6]:
ligand=mdtraj.load(ligand_pdb_file)
def create_box_from_ligand(ligand):
    xyz=ligand.xyz[0]*10 # convert to Angstrom from nm
    pocket_center = (xyz.max(axis=0) + xyz.min(axis=0)) / 2
    pocket_size = xyz.max(axis=0) - xyz.min(axis=0) + 5
    return Box.from_array(pocket_center, pocket_size)

box=create_box_from_ligand(ligand)
box

Box(center=Point(x=-51.568, y=0.7765, z=23.5065), size=Point(x=11.748001, y=12.989, z=17.699))

Get the docker

In [7]:
from src.workshop_2_utils import Docking 
    
docker=Docking(protein_pdbqt_file, box, num_poses=num_poses, exhaustiveness=exhaustiveness) 


`pdbqt` for the molecules

In [8]:
import datamol as dm 

df_mols = dm.read_sdf(all_ligands_sdf_file, as_df=True, mol_column="mols", n_jobs=-1)


Dock them!

In [9]:
if num_mols > len(df_mols):
    num_mols = len(df_mols)

docker.dock_multiple_mols(
        df_mols["mols"].tolist()[:num_mols],
        input_dir = smina_inputs_path,
        output_dir = smina_outputs_path,
        idxs= list(range(num_mols)),
)      

2024-06-20 18:29:37.315 | INFO     | src.workshop_2_utils:dock_multiple_mols:266 - Converting mols to pdbqt in '/home/ubuntu/smina/ml4dd/data/lab2/smina_inputs' folder
2024-06-20 18:29:37.325 | INFO     | src.workshop_2_utils:dock_multiple_mols:270 - Docking
100%|██████████| 5/5 [00:21<00:00,  4.22s/it]
2024-06-20 18:29:58.431 | INFO     | src.workshop_2_utils:dock_multiple_mols:275 - Merge all the generated poses together to /home/ubuntu/smina/ml4dd/data/lab2/smina_outputs


Read the poses

In [10]:
poses= dm.read_sdf(smina_outputs_path / "poses.sdf", as_df=True, mol_column="mols", n_jobs=-1)
poses.sort_values("minimizedAffinity",inplace=True)

Need to get rid of duplicates!

In [11]:
from rdkit import Chem

# Function to convert a molecule to its canonical SMILES
def mol_to_canonical_smiles(mol):
    return Chem.MolToSmiles(Chem.MolFromSmiles(mol), isomericSmiles=True)

# Apply the function to each molecule in the DataFrame
poses['canonical_smiles'] = poses['smiles'].apply(mol_to_canonical_smiles)

# Drop duplicates based on the 'canonical_smiles' column
poses.drop_duplicates(subset='canonical_smiles', inplace=True)


In [12]:
poses

,smiles,mols,minimizedAffinity,canonical_smiles
114,Cc1csc(Nc2ccccc2)n1,<rdkit.Chem.rdchem.Mol object at 0x7f1e018e5c40>,-6.285374,Cc1csc(Nc2ccccc2)n1
99,Nc1nc2c(F)c(F)c(F)cc2s1,<rdkit.Chem.rdchem.Mol object at 0x7f1e018e55b0>,-6.238235,Nc1nc2c(F)c(F)c(F)cc2s1
59,Nc1nc2c(Cl)c(Cl)ccc2s1,<rdkit.Chem.rdchem.Mol object at 0x7f1e018e4430>,-6.002104,Nc1nc2c(Cl)c(Cl)ccc2s1
17,NC(=O)/C=[S]/[C@H]1[N]C(N)=NC(N)=N1,<rdkit.Chem.rdchem.Mol object at 0x7f1e01c2f370>,-5.575815,NC(=O)/C=[S]/[C@H]1[N]C(N)=NC(N)=N1
54,NC(=O)/C=[S]/[C@@H]1[N]C(N)=NC(N)=N1,<rdkit.Chem.rdchem.Mol object at 0x7f1e018e4200>,-5.571984,NC(=O)/C=[S]/[C@@H]1[N]C(N)=NC(N)=N1
94,c1ccc(Nc2nccs2)nc1,<rdkit.Chem.rdchem.Mol object at 0x7f1e018e5380>,-5.552175,c1ccc(Nc2nccs2)nc1


In [13]:
poses.drop(columns=['mols']).to_csv(save_df_file)